# OSMnx + igraph

Author: [Geoff Boeing](https://geoffboeing.com/)

First install [igraph](https://igraph.org/python/) or run Jupyter from the [Docker container](https://hub.docker.com/r/gboeing/osmnx) (which already has it installed along with OSMnx and NetworkX).

  - [Overview of OSMnx](http://geoffboeing.com/2016/11/osmnx-python-street-networks/)
  - [GitHub repo](https://github.com/gboeing/osmnx)
  - [Examples, demos, tutorials](https://github.com/gboeing/osmnx-examples)
  - [Documentation](https://osmnx.readthedocs.io/en/stable/)
  - [Journal article/citation](http://geoffboeing.com/publications/osmnx-complex-street-networks/)

In [1]:
import igraph as ig
import networkx as nx
import numpy as np
import operator
import osmnx as ox
%matplotlib inline
ox.config(use_cache=True, log_console=True)
weight = 'length'

ox.__version__

'0.16.0'

## Construct graphs

In [2]:
# create networkx graph
G_nx = ox.graph_from_place('Piedmont, CA, USA', network_type='drive')
G_nx = nx.relabel.convert_node_labels_to_integers(G_nx)

In [3]:
%%time
# convert networkx graph to igraph
G_ig = ig.Graph(directed=True)
G_ig.add_vertices(list(G_nx.nodes()))
G_ig.add_edges(list(G_nx.edges()))
G_ig.vs['osmid'] = list(nx.get_node_attributes(G_nx, 'osmid').values())
G_ig.es[weight] = list(nx.get_edge_attributes(G_nx, weight).values())

CPU times: user 4.26 ms, sys: 29 µs, total: 4.29 ms
Wall time: 14.7 ms


In [4]:
assert len(G_nx.nodes()) == G_ig.vcount()
assert len(G_nx.edges()) == G_ig.ecount()

## Shortest paths

In [5]:
source = list(G_nx.nodes())[0]
target = list(G_nx.nodes())[-1]

In [6]:
%%time
path1 = G_ig.get_shortest_paths(v=source, to=target, weights=weight)[0]

CPU times: user 160 µs, sys: 19 µs, total: 179 µs
Wall time: 526 µs


In [7]:
%%time
path2 = nx.shortest_path(G_nx, source, target, weight=weight)

CPU times: user 225 µs, sys: 26 µs, total: 251 µs
Wall time: 255 µs


In [8]:
assert path1 == path2

In [9]:
%%time
path_length1 = G_ig.shortest_paths(source=source, target=target, weights=weight)[0][0]

CPU times: user 90 µs, sys: 11 µs, total: 101 µs
Wall time: 105 µs


In [10]:
%%time
path_length2 = nx.shortest_path_length(G_nx, source, target, weight)

CPU times: user 191 µs, sys: 23 µs, total: 214 µs
Wall time: 217 µs


In [11]:
assert path_length1 == path_length2

## Centrality analysis

In [12]:
%%time
closeness1 = G_ig.closeness(vertices=None, mode='ALL', cutoff=None, weights=weight, normalized=True)
max_closeness1 = np.argmax(closeness1)

CPU times: user 31 ms, sys: 345 µs, total: 31.4 ms
Wall time: 32.7 ms


In [13]:
%%time
closeness2 = nx.closeness_centrality(G_nx, distance=weight, wf_improved=True)
max_closeness2 = max(closeness2.items(), key=operator.itemgetter(1))[0]

CPU times: user 422 ms, sys: 3.56 ms, total: 425 ms
Wall time: 428 ms


In [14]:
assert G_nx.nodes[max_closeness2]['osmid'] == G_ig.vs[max_closeness1]['osmid']